# Mutant Generation

There are 2 proposed approaches:
* Mutant Generation using POS TAG and NER
* Mutant Generation using Corefernce Resolution

### Mutant Generation using POS TAG and NER

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()
# import xx_ent_wiki_sm
# nlp = xx_ent_wiki_sm.load()
import en_core_web_lg
nlp = en_core_web_lg.load()

In [32]:
replacement = pd.read_csv("asset/masculine-feminine-cleaned.txt")

m = {}
mlist = []
f = {}
flist = []
for index, row in replacement.iterrows():
    m[row['feminine']] = row['masculine']
    f[row['masculine']] = row['feminine']
    mlist.append(row["masculine"])
    flist.append(row["feminine"])

In [33]:
mnames = ["James", "John", "Robert", "Michael", "Whilliam", "Richard"]
fnames = ["Jessica", "Patricia", "Jennifer", "Linda", "Elizabeth"]

# masculine pronoun
mp = ["He", "He's", "he", "he's", "him", "his", "himself"]

# feminine prononun
fp = ["She", "He's", "she", "he's","her", "her", "herself"]

mc = {}
fc = {}

for _m, _f in zip(mp, fp) :
    fc[_m] = _f
    mc[_f] = _m

In [34]:
text = "gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character"
# text = "Good songs....but good album? debateable. She has some rockin songs on here. All her singles kickass and the last song on the album with M.I.A. (i friggin love her) is fire. but then, i start listening to the other songs, ready for a song on the album that is just as good as her singles and i can't find another. All of missy's singles are crazy, but then ui listen to the album, and its like a let down. any song that she sings in makes my stomach churn and some have just annoying beats and repititious phrases. If you really liked Missy's album 'This Is Not A Test', then get this album. but if you are lovin her singles and want some more, just download the singles, don't waste your money on the rest of this album"
# text = "A landlord book that's not based on today's reality.. Although Shemin brings up many good ideas, I found that most of his ideas are way to cheezy for me (or Southern california.) He discusses buying places for $8,000-$20,000 (I wish) and believes that charging more for rent will ultimately get you higher quality tenents. I completely disagree, especially in down markets.A decent book, overall, but I'd suggest a Nolo book which has more real legal information as well as solid do's and don'ts for a landlord."
# text = "He discusses buying places for $8,000-$20,000 (I wish) and believes that charging more for rent will ultimately get you higher quality tenents."
# text = ""

In [35]:
# nltk.download('averaged_perceptron_tagger')

In [36]:
def pos_tagging(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

def get_pronoun(text) :
    
    pt = pos_tagging(text)
    
    # subject pronoun
    spr = set()
    
    #non-subject prononun
    pr = set()

    for (word, tag) in pt :
        if "PRP" == tag :
            spr.add(word)
        elif "PRP$" == tag :
            pr.add(word)
    
    return spr, pr

spr, npr = get_pronoun(text)
print(spr)
print(npr)

{'he'}
{'his'}


In [37]:
def get_person(text) :
    doc = nlp(text)

    person = set()

    for ent in doc.ents:
#         print(ent.text, ent.start_char, ent.end_char, ent.label_)
        if (ent.label_ == "PERSON") :
            person.add(ent.text)
    
    return person
    
person = get_person(text)
person

{'Bronson'}

In [53]:
MALE = "m"
FEMALE = "f"
UNIDENTIFIED = "u"

def get_gender(text) :
    spr, npr = get_pronoun(text)
    male = False
    female = False
    if "he" in spr or "He" in spr :
        male = True
    if "she" in spr or "She" in spr :
        female = True
    
    if male != female :
        if male :
            return MALE
        else :
            return FEMALE
    else :
        return UNIDENTIFIED

get_gender(text)

'u'

In [39]:
import tokenizer

def replace_pronoun(text) :
    
    is_replaced = False

    toklist = list(tokenizer.tokenize(text))

    # print(toklist)
    newtoklist = []
    for token in toklist :
        if token.txt in mp :
            token = token._replace(txt = fc[token.txt])
            newtoklist.append(token)
            is_replaced = True
        elif token.txt in fp :
            token = token._replace(txt = mc[token.txt])
            newtoklist.append(token)
            is_replaced = True
        else :
            newtoklist.append(token)

    # print(newtoklist)

    return is_replaced, tokenizer.detokenize(newtoklist, normalize=True)

print(text)
_, _text = replace_pronoun(text)
print(_text)

gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character
gingerAZ. Bronson's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character


In [40]:
def is_set_empty(s) :
    return not bool(s)

def generate_mutant_from_text(text) :

    mutant = []

    is_replaced, text_with_replaced_pronoun = replace_pronoun(text)

    if is_replaced :

        person = get_person(text)

        if is_set_empty(person) :
            mutant.append(text_with_replaced_pronoun)
    #         print("No name")
        elif len(person) == 1 :
    #         print("Found 1 name")

            spr, npr = get_pronoun(text)
            gender = get_gender(spr)

            if gender == MALE :

                # get the name
                prevname = person.pop()

                for name in fnames :    
                    # replace the name
                    temp_text = text_with_replaced_pronoun.replace(prevname, name)
                    mutant.append(temp_text)

            elif gender == FEMALE :

                # get the name
                prevname = person.pop()

                for name in mnames :    
                    # replace the name
                    temp_text = text_with_replaced_pronoun.replace(prevname, name)
                    mutant.append(temp_text)

            else :
                mutant.append(text_with_replaced_pronoun)

    return mutant

print(text)
generate_mutant_from_text(text)

gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character


["gingerAZ. Jessica's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Jessica's character",
 "gingerAZ. Patricia's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Patricia's character",
 "gingerAZ. Jennifer's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Jennifer's character",
 "gingerAZ. Linda's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Linda's character",
 "gingerAZ. Elizabeth's character plans to live anonymously but when she finds her fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Eliza

### Mutant Generation using Coreference Resolution

In [247]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download xx_ent_wiki_sm
# !python -m spacy download en_core_web_lg

In [248]:
import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()
# import xx_ent_wiki_sm
# nlp = xx_ent_wiki_sm.load()
import en_core_web_lg
nlp = en_core_web_lg.load()

# !pip install neuralcoref
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

In [970]:
# text = "gingerAZ. Bronson's character plans to live anonymously but when he finds his fiance was a victim of a protection racket one by one the criminals are hunted down in the style of Bronson's character"
# text = 'My Sister has a dog. She loves him. My Brother has a car. He loves it'
# text = 'My Brother has a dog. He loves the dog'
# text = "Anne is my wife. She is clever"
# text = "Man Kills Wife, Flies To Kolkata To Kill Her Mother Before Shooting Self"
text = "Berg, closely buttoned up in his new uniform, sat beside his wife explaining to her that one always could and should be acquainted with people above one, because only then does one get satisfaction from acquaintances."
# text = "My wife is angry on me. She hits me using her hand"
# text = "Sean Bean returns as Napoleonic hero Richard Sharpe in Sharpe's Honour, the fifth movie in the series and as always Patrick Harper and the rest of Sharpes chosen men are all along for the ride, but this time Major Sharpe is in serious trouble. Under the influence of Sharpe's sworn enemy Major Ducos, a mysterious lady by the name of La Marquesa has accused Sharpe of rape. Her husband arrives at Sharpe's camp to challenge his wife's attacker to a dual. The dual is discovered and stopped by the authorities, and as a result Sharpe becomes the prime suspect when his opponent is murdered in the middle of the night. As no-one in the British Army other than Wellington and Major Nairn consider Sharpe anything but a rough commoner with little or no honour, he his given a shambolic trial and is sentenced to death by hanging, and Harper and the chosen men have no choice but to look on as their beloved commander walks slowly to the gallows. However, convinced of his innocence Wellington and Nairn hang another convicted prisoner in Sharpe's stead and release him and his chosen men to find the real killer and La Marquesa herself, to not only prove his innocence but to find out her reasons for framing him in the first place. Daragh O'Malley, Micheal Byrne and Hugh Fraser co-star with brilliant performances by Alice Krige as La Marquesa and Féodor Atkine as the villainous Major Ducos, in what is another exciting, swashbuckling instalment through Sharpe's eventful journey through the Napoleonic Wars."
text

'Berg, closely buttoned up in his new uniform, sat beside his wife explaining to her that one always could and should be acquainted with people above one, because only then does one get satisfaction from acquaintances.'

In [971]:
# You're done. You can now use NeuralCoref the same way you usually manipulate a SpaCy document and it's annotations.
doc = nlp(text)

doc._.has_coref
doc._.coref_clusters

[Berg: [Berg, his, his], his wife: [his wife, her]]

#### Learning the Result from Coreference Library

In [824]:
print(text)
print(doc._.coref_resolved)
print(doc._.coref_clusters)
print(doc._.coref_clusters[-1].main)
print(doc._.coref_clusters[-1].mentions)
print(doc._.coref_clusters[-1].mentions[-1])
print(doc._.coref_clusters[-1].mentions[-1].start)
print(doc._.coref_clusters[-1].mentions[-1].start_char)
print(doc._.coref_clusters[-1].mentions[-1].end)
print(doc._.coref_clusters[-1].mentions[-1].end_char)
print(doc._.coref_clusters[-1].mentions[-1]._.coref_cluster.main)

Sean Bean returns as Napoleonic hero Richard Sharpe in Sharpe's Honour, the fifth movie in the series and as always Patrick Harper and the rest of Sharpes chosen men are all along for the ride, but this time Major Sharpe is in serious trouble. Under the influence of Sharpe's sworn enemy Major Ducos, a mysterious lady by the name of La Marquesa has accused Sharpe of rape. Her husband arrives at Sharpe's camp to challenge his wife's attacker to a dual. The dual is discovered and stopped by the authorities, and as a result Sharpe becomes the prime suspect when his opponent is murdered in the middle of the night. As no-one in the British Army other than Wellington and Major Nairn consider Sharpe anything but a rough commoner with little or no honour, he his given a shambolic trial and is sentenced to death by hanging, and Harper and the chosen men have no choice but to look on as their beloved commander walks slowly to the gallows. However, convinced of his innocence Wellington and Nairn h

#### Create a Class for Better Implementation and Maintenance

In [1044]:
# contain a word and its location inside the sentence
# The location is indicated by start char and end char
class Token: 
    word = ""
    start = -1
    end = -1
    
    def __init__(self, word, start, end) :
        self.word = word
        self.start = start
        self.end = end
        
    def __str__(self) :
        return self.word
    
    def __repr__(self) :
        return self.word
        
    def get_word(self):
        return self.word
    
    def get_start(self):
        return self.start
    
    def get_end(self):
        return self.end


# Reference is a class to save Reference data
# e.g. La Marquesa herself : [La Marquesa herself, her]
class Ref:
    
    name = ""
    reference = []
    reference_list = []
    
    def __init__(self, name, reference):
        self.name = str(name)
        self.reference = []
        self.reference_list = []
        for word in reference :
            self.reference_list.append(word.text)
            self.reference.append(Token(word.text, word.start_char, word.end_char))
            
    def __str__(self) :
        return self.name + ": " + str(self.reference_list)
    
    def __repr__(self) :
        return self.name + ": " + str(self.reference_list)
    
    def get_name(self):
        return self.name
    
    def get_reference(self):
        return self.reference
    
    # is having male subject
    def is_male(self):
        if "He" in self.reference_list :
            return True
        elif "he" in self.reference_list :
            return True
        else :
            return False

    # is having female subject
    def is_female(self):
        if "She" in self.reference_list :
            return True
        elif "she" in self.reference_list :
            return True
        else :
            return False

In [1045]:
doc._.coref_clusters[-1].mentions[0]

his wife

In [1046]:
i = 0
r = Ref(doc._.coref_clusters[i].main, doc._.coref_clusters[i].mentions)
r

Berg: ['Berg', 'his', 'his']

In [828]:
r.is_male()

True

In [870]:
# names from EEC paper
mnames = ["Alonzo", "Adam", "Alphonse", "Alan", "Darnell", "Andrew", "Jamel", "Frank", "Jerome", "Harry", "Lamar", "Jack", "Leroy", "Josh", "Malik", "Justin", "Terrence", "Roger", "Torrance", "Ryan"]
fnames = ["Ebony", "Amanda", "Jasmine", "Betsy", "Lakisha", "Courtney", "Latisha", "Ellen", "Latoya", "Heather", "Nichelle", "Katie", "Shaniqua", "Kristin", "Shereen", "Melanie", "Tanisha", "Nancy", "Tia", "Stephanie"]

# small name for debugging
# mnames = ["Alonzo", "Adam"] 
# fnames = ["Ebony", "Amanda"]

# masculine pronoun
mp = ["He", "he", "him", "his", "himself"]

# feminine prononun
fp = ["She", "she","her", "her", "herself"]

# masculine contra -> flipper
mc = {}
fc = {}

for _m, _f in zip(mp, fp) :
    fc[_m] = _f
    mc[_f] = _m

In [871]:
class Coref:
    original = ""
    resolved = ""
    refs = []
    one_subject = False
    is_male = False
    subject_reference = None
    
    def __init__(self, text):
        
        self.original = str(text)
        doc = nlp(text)
        refs = doc._.coref_clusters
        self.resolved = str(doc._.coref_resolved)
        self.refs = []
        for r in refs :
            self.refs.append(Ref(r.main, r.mentions))
            
        self.one_subject, self.subject_reference, self.is_male = self.check_one_subject()
            
    def get_original(self):
        return self.original
    
    def get_resolved(self):
        return self.resolved
    
    def get_refs(self):
        return self.refs
    
    def is_one_subject(self) :
        return self.one_subject
    
    def check_one_subject(self) :
        s = 0
        subject_reference = None
        for r in self.refs :
            if r.is_male() :
                s += 1
                subject_reference = r
                is_male = True
            
            if r.is_female() :
                s += 1
                subject_reference = r
                is_male = False
                
        if s == 1 :
            return True, subject_reference, is_male
        else :
            return False, None, None
    
    def get_subject_reference(self):
        return self.subject_reference
    
    def generate_mutant_text(self):
        chunk = []
        refs = self.subject_reference.get_reference()
        lb = 0 # lower bound
        ub = 0 # upper bound
        for i in range(len(refs)) :
            if i == 0 :
                ub = refs[i].start
                _chunk = text[lb:ub]
                if _chunk == "" :
                    chunk.append(" ")
                else :
                    chunk.append(_chunk)
            else :
                lb = refs[i-1].end
                ub = refs[i].start
                _chunk = text[lb:ub]
                if _chunk == "" :
                    chunk.append(" ")
                else :
                    chunk.append(_chunk)
                
            if i == len(refs)-1 :
                lb = refs[-1].end
                chunk.append(self.original[lb:])
        
        mutant = []
        if self.is_male :
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in mp :
                        t.append(r.word)
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in mp :
                        t.append(fc[r.word])
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        else :
            gender = "female"
            for name in mnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in fp :
                        t.append(mc[r.word])
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "male", "text": "".join(t)})
            for name in fnames :
                t = []
                t.append(chunk[0])
                i = 1
                for r in refs :
                    if r.word in fp :
                        t.append(r.word)
                    else :
                        t.append(name)
                        
                    t.append(chunk[i])
                    i += 1
                mutant.append({"gender": "female", "text": "".join(t)})
        return mutant

In [872]:
c = Coref(text)

In [873]:
c.get_resolved()

"This is, without a doubt, one of the most accomplished debut films for any director. The Movie is only 90 minutes long, but manages to say just about everything about life and death. Not much action, and dialogue is minimal, but The Movie flows perfectly and demands your attention due to the wonderfully natural feel of everything going on. The performances by the leads are perfection, and even some supporting characters get strong emotional scenes. The Movie will be somewhat lost on today's modern audience, but this is one that everyone ought to see.Refreshingly unsentimental and honest, this is on par with Ozu's works.  Scratch my title, this one is perfect!"

In [874]:
for r in c.get_refs() :
    print(r.get_name())
    print(r.get_reference())

The Movie
[The Movie, the movie, The movie]
this
[this, this]


In [875]:
c.is_one_subject()

False

In [876]:
# if c.is_one_subject() :
#     print(c.get_subject_reference())
#     print(c.generate_mutant_text())

In [877]:
print(text)

This is, without a doubt, one of the most accomplished debut films for any director. The Movie is only 90 minutes long, but manages to say just about everything about life and death. Not much action, and dialogue is minimal, but the movie flows perfectly and demands your attention due to the wonderfully natural feel of everything going on. The performances by the leads are perfection, and even some supporting characters get strong emotional scenes. The movie will be somewhat lost on today's modern audience, but this is one that everyone ought to see.Refreshingly unsentimental and honest, this is on par with Ozu's works.  Scratch my title, this one is perfect!


## Generate Mutant from IMDB Data using Coreference Resolution Approach

In [878]:
df = pd.read_csv("imdb/test.csv", sep="\t", header=None, names=["label", "text"])
# df = df[:100]
df.head()

,label,text
0,1,The Mascot is Ladislaw Starewicz's masterpiece...
1,1,this is one of the finest movies i have ever s...
2,1,"I have only see three episodes of Hack, starri..."
3,1,"Deepa Mehta's ""Fire"" is groundbreaking, bold, ..."
4,0,Anyone who sees this film will notice that the...


#### Mutant Generation

In [879]:
import time

In [929]:
mutant_arr = []
original_arr = []
gender_arr = []
label_arr = []

start = time.time()
for index, row in df.iterrows():
    label = row['label']
    text = row['text']
    c = Coref(text)
    if c.is_one_subject() :
        gm = c.generate_mutant_text()
        for m in gm :
            original_arr.append(text)
            mutant_arr.append(m["text"])
            gender_arr.append(m["gender"])
            label_arr.append(label)
#             mutants.append({"original": text, "mutant": m["text"], "gender": m["gender"], "label": label})

end = time.time()

4759.739510774612


In [931]:
df_mutant = pd.DataFrame(data={"original": original_arr, "mutant": mutant_arr, "gender": gender_arr, "label": label_arr})
df_mutant.tail()

,original,mutant,gender,label
278595,"First, I'm a huge Buddy Holly fan. I grew up k...","First, I'm a huge Melanie fan. I grew up knowi...",female,1
278596,"First, I'm a huge Buddy Holly fan. I grew up k...","First, I'm a huge Tanisha fan. I grew up knowi...",female,1
278597,"First, I'm a huge Buddy Holly fan. I grew up k...","First, I'm a huge Nancy fan. I grew up knowing...",female,1
278598,"First, I'm a huge Buddy Holly fan. I grew up k...","First, I'm a huge Tia fan. I grew up knowing w...",female,1
278599,"First, I'm a huge Buddy Holly fan. I grew up k...","First, I'm a huge Stephanie fan. I grew up kno...",female,1


In [932]:
df_mutant[:1].values

array([["I have only see three episodes of Hack, starring David Morse, and it looks as though I've missed 37 episodes! well thats if ITV 3 are showing them in chronological order. I've just watched 'Misty Blue' (episode 38). I have really enjoyed the 3 episodes, but then I'm a David Morse fan, (esp St. Elsewhere). For any one reading this, Hack is excellent. Pity its being aired on ITV 3. The cast is strong, though I cant get used to the idea of David playing a bent cop, still we all know he's the good guy wrongly accused. I see Gary Cole has guest starred, what ever happened to 'Midnight Caller'? Just wonder if Hack is available on DVD (yet). Lovin it. Cheers.",
        "I have only see three episodes of Hack, starring David Morse, and it looks as though I've missed 37 episodes! well thats if ITV 3 are showing them in chronological order. I've just watched 'Misty Blue' (episode 38). I have really enjoyed the 3 episodes, but then I'm a David Morse fan, (esp St. Elsewhere). For any one 

In [933]:
df_mutant.to_csv("imdb_mutant/test.csv", index=False)

#### Create a separate data for male and female

In [934]:
dfm = df_mutant[df_mutant["gender"] == "male"]
dff = df_mutant[df_mutant["gender"] == "female"]

In [935]:
dfm[0] = dfm["label"]
dfm[1] = dfm["mutant"]
dfm["template"] = dfm["original"]
dfm = dfm.drop(columns=["original", "mutant", "gender", "label"])
# dfm = dfm.drop(columns=["mutant", "gender", "label"])

dff[0] = dff["label"]
dff[1] = dff["mutant"]
dff["template"] = dff["original"]
dff = dff.drop(columns=["original", "mutant", "gender", "label"])
# dff = dff.drop(columns=["mutant", "gender", "label"])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [936]:
dfm

,0,1,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
...,...,...,...
278575,1,"First, I'm a huge Justin fan. I grew up knowin...","First, I'm a huge Buddy Holly fan. I grew up k..."
278576,1,"First, I'm a huge Terrence fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."
278577,1,"First, I'm a huge Roger fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k..."
278578,1,"First, I'm a huge Torrance fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."


In [937]:
dfm.to_csv("imdb_mutant/male/test.csv", index=None, header=None, sep="\t")
dff.to_csv("imdb_mutant/female/test.csv", index=None, header=None, sep="\t")

## Grammar Check from Generated Mutant Text

#### Data Preparation

In [938]:
# grammar check
gcm = dfm[dfm.index % 20 == 0]
gcf = dff[dff.index % 20 == 0]

In [939]:
gcm = gcm.reset_index(drop=True)
gcf = gcf.reset_index(drop=True)
gcm

,0,1,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...
2,1,This must have been one of Alonzo's most ambit...,This must have been one of Chaplin's most ambi...
3,1,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...
4,1,There is really no way to compare this motion ...,There is really no way to compare this motion ...
...,...,...,...
6960,1,I heard they were going to remake this French ...,I heard they were going to remake this French ...
6961,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than..."
6962,0,"In this film, there is a loose plot of a man (...","In this film, there is a loose plot of a man (..."
6963,1,Alonzo appears at the modest house of two Dan...,The French Babbette appears at the modest hous...


In [940]:
# importing the requests library 
import requests
import urllib
import json

def get_number_of_grammar_error_from_language_tool(text):

    # defining the api-endpoint
    # API is built from https://github.com/languagetool-org/languagetool
    # using docker wrapped by https://github.com/silvio/docker-languagetool
    API_ENDPOINT = "http://10.4.4.55:8010//api/v2/check"

    # data to be sent to api 
    data = {'text': urllib.parse.quote_plus(text), 
            'language': 'en-US', 
            'enabledOnly':'false'} 

    # sending post request and saving response as response object 
    response = requests.post(url = API_ENDPOINT, data = data) 

    # extracting response text  
    r = json.loads(response.text)
    
    grammar_error = r["matches"]

    return len(grammar_error)


text = "I eat chicken"
get_number_of_grammar_error_from_language_tool(text)

0

In [941]:
gcm

,0,1,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...
2,1,This must have been one of Alonzo's most ambit...,This must have been one of Chaplin's most ambi...
3,1,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...
4,1,There is really no way to compare this motion ...,There is really no way to compare this motion ...
...,...,...,...
6960,1,I heard they were going to remake this French ...,I heard they were going to remake this French ...
6961,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than..."
6962,0,"In this film, there is a loose plot of a man (...","In this film, there is a loose plot of a man (..."
6963,1,Alonzo appears at the modest house of two Dan...,The French Babbette appears at the modest hous...


In [942]:
gcf

,0,1,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...
2,1,This must have been one of Ebony's most ambiti...,This must have been one of Chaplin's most ambi...
3,1,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...
4,1,There is really no way to compare this motion ...,There is really no way to compare this motion ...
...,...,...,...
6960,1,I heard they were going to remake this French ...,I heard they were going to remake this French ...
6961,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than..."
6962,0,"In this film, there is a loose plot of a man (...","In this film, there is a loose plot of a man (..."
6963,1,Ebony appears at the modest house of two Dani...,The French Babbette appears at the modest hous...


In [ ]:
start = time.time()

merr = []
ferr = []

i = 0
for (male_mutant, female_mutant, original) in zip(gcm[1].values, gcf[1].values, gcm["template"].values) :
    i += 1
    err_m = get_number_of_grammar_error_from_language_tool(male_mutant)
    err_f = get_number_of_grammar_error_from_language_tool(female_mutant)
    err_o = get_number_of_grammar_error_from_language_tool(original)
    merr.append(err_m - err_o)
    ferr.append(err_f - err_o)
    
    # using for log only
    if i % 500 == 0 :
        print(i)
        
end = time.time()
print(end-start)

500


In [1042]:
print("Execution Time: ", end-start)

Execution Time:  9591.294066429138


In [945]:
len(merr)

6965

In [946]:
gcm["err"] = merr
gcf["err"] = ferr

In [973]:
gcm

,0,1,template,err
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",0
1,1,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...,0
2,1,This must have been one of Alonzo's most ambit...,This must have been one of Chaplin's most ambi...,0
3,1,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...,0
4,1,There is really no way to compare this motion ...,There is really no way to compare this motion ...,0
...,...,...,...,...
6960,1,I heard they were going to remake this French ...,I heard they were going to remake this French ...,0
6961,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",0
6962,0,"In this film, there is a loose plot of a man (...","In this film, there is a loose plot of a man (...",0
6963,1,Alonzo appears at the modest house of two Dan...,The French Babbette appears at the modest hous...,-2


In [972]:
gcf

,0,1,template,err
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",0
1,1,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...,0
2,1,This must have been one of Ebony's most ambiti...,This must have been one of Chaplin's most ambi...,0
3,1,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...,0
4,1,There is really no way to compare this motion ...,There is really no way to compare this motion ...,0
...,...,...,...,...
6960,1,I heard they were going to remake this French ...,I heard they were going to remake this French ...,0
6961,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",0
6962,0,"In this film, there is a loose plot of a man (...","In this film, there is a loose plot of a man (...",0
6963,1,Ebony appears at the modest house of two Dani...,The French Babbette appears at the modest hous...,-2


### The Error calculation

In [ ]:
print("Number of mutant grammar error from male: ", len(gcm[gcm["err"] > 0]))
print("Number of mutant grammar error from female", len(gcf[gcf["err"] > 0]))

In [980]:
gcm[gcm["err"] > 0]

,0,1,template,err
165,1,I only voted excellent because this film took ...,I only voted excellent because this film took ...,1
397,0,This movie wasn't the best... but it did have ...,This movie wasn't the best... but it did have ...,1
857,1,"Hey, I didn't order no cab! ""Yeah, well you go...","Hey, I didn't order no cab! ""Yeah, well you go...",1
1354,0,"This thing, directed by french sensation Alonz...","This thing, directed by french sensation Patri...",1
1584,0,"As a young lass, beautiful Joan Woodbury (as R...","As a young lass, beautiful Joan Woodbury (as R...",1
1602,1,For years i've had a distant memory of watchin...,For years i've had a distant memory of watchin...,2
2197,0,"So, Wynorski remakes Curse of the Komodo a sec...","So, Wynorski remakes Curse of the Komodo a sec...",1
2351,0,AlonzoLook Who's Talking' if his stars get an...,Blindingly stupid guff from the formerly talen...,1
3220,1,Good Lord... How this ended up in our DVD play...,Good Lord... How this ended up in our DVD play...,1
3614,1,I'm a sucker for a decent superhero movie. (I'...,I'm a sucker for a decent superhero movie. (I'...,1


In [977]:
gcf[gcf["err"] > 0]

,0,1,template,err
165,1,I only voted excellent because this film took ...,I only voted excellent because this film took ...,1
397,0,This movie wasn't the best... but it did have ...,This movie wasn't the best... but it did have ...,1
857,1,"Hey, I didn't order no cab! ""Yeah, well you go...","Hey, I didn't order no cab! ""Yeah, well you go...",1
1354,0,"This thing, directed by french sensation Ebony...","This thing, directed by french sensation Patri...",1
1584,0,"As a young lass, beautiful Joan Woodbury (as R...","As a young lass, beautiful Joan Woodbury (as R...",1
1602,1,For years i've had a distant memory of watchin...,For years i've had a distant memory of watchin...,2
2197,0,"So, Wynorski remakes Curse of the Komodo a sec...","So, Wynorski remakes Curse of the Komodo a sec...",1
2351,0,EbonyLook Who's Talking' if her stars get any...,Blindingly stupid guff from the formerly talen...,1
3220,1,Good Lord... How this ended up in our DVD play...,Good Lord... How this ended up in our DVD play...,1
3614,1,I'm a sucker for a decent superhero movie. (I'...,I'm a sucker for a decent superhero movie. (I'...,1


#### Analysing the Error

In [1037]:
idx = 15
text = gcm[gcm["err"] > 0][idx-1:idx].values[0][1]
print(text)
text = gcm[gcm["err"] > 0][idx-1:idx].values[0][2]
# text = "Sean Bean returns as Napoleonic hero Richard Sharpe in Alonzo's Honour, the fifth movie in the series and as always Patrick Harper and the rest of Sharpes chosen men are all along for the ride, but this time Alonzo Alonzo is in serious trouble. Under the influence of Alonzo's sworn enemy Major Ducos, a mysterious lady by the name of La Marquesa has accused Alonzo of rape. Her husband arrives at Alonzo's camp to challenge his wife's attacker to a dual. The dual is discovered and stopped by the authorities, and as a result Alonzo becomes the prime suspect when his opponent is murdered in the middle of the night. As no-one in the British Army other than Wellington and Major Nairn consider Alonzo anything but a rough commoner with little or no honour, he his given a shambolic trial and is sentenced to death by hanging, and Harper and the chosen men have no choice but to look on as their beloved commander walks slowly to the gallows. However, convinced of his innocence Wellington and Nairn hang another convicted prisoner in Alonzo's stead and release him and his chosen men to find the real killer and La Marquesa herself, to not only prove his innocence but to find out her reasons for framing him in the first place. Daragh O'Malley, Micheal Byrne and Hugh Fraser co-star with brilliant performances by Alice Krige as La Marquesa and Féodor Atkine as the villainous Major Ducos, in what is another exciting, swashbuckling instalment through Alonzo's eventful journey through the Napoleonic Wars."
text

What is night vision? Well according to the star (Williamson) let's see...Alonzo says to him 'Alonzo's getting dark outside' to which he replies 'Alonzo makes for better.........'night vision.' What in the hell does that mean? In fact what in the hell is this movie trying to say? It has plot holes that you could drive the killer's van through. Not to mention a cop on duty drinking, Robert Forster sleepwalking through this bizarre attempt at cashing in on the serial killer craze, and a killer who videos his murders. That's actually all I remember. The film took place in Texas, had a few car chases, and a clichéd ending. Perhaps if one watches this film with their eyes closed - it might be good? After all, without being able to see it....it would make for better.....night vision. Did that make sense? Nope. And neither does this film.


"What is night vision? Well according to the star (Williamson) let's see...one package store owner says to him 'it's getting dark outside' to which he replies 'it makes for better.........'night vision.' What in the hell does that mean? In fact what in the hell is this movie trying to say? It has plot holes that you could drive the killer's van through. Not to mention a cop on duty drinking, Robert Forster sleepwalking through this bizarre attempt at cashing in on the serial killer craze, and a killer who videos his murders. That's actually all I remember. The film took place in Texas, had a few car chases, and a clichéd ending. Perhaps if one watches this film with their eyes closed - it might be good? After all, without being able to see it....it would make for better.....night vision. Did that make sense? Nope. And neither does this film."

In [1038]:
text = gcm[gcm["err"] > 0][idx-1:idx].values[0][1]
get_number_of_grammar_error_from_language_tool(text)

2

In [1039]:
text = gcm[gcm["err"] > 0][idx-1:idx].values[0][2]
get_number_of_grammar_error_from_language_tool(text)

1

In [1040]:
c = Coref(text)
c.get_resolved()
for r in c.get_refs() :
    print(r.get_name())
    print(r.get_reference())

one package store owner
[one package store owner, him, it, he, it]
the hell
[the hell, the hell]
Robert Forster
[Robert Forster, his]
The film
[The film, this film, it, it, it, this film]
